In [57]:
from abc import ABC, abstractmethod
from enum import Enum
from datetime import date 
from dateutil.easter import *

In [31]:
from enum import Enum
from typing import Union
from datetime import timedelta


class TimeUnit(Enum):
    Days = 1
    Weeks = 7
    Months = 30  # Approximation for a month
    Years = 365  # Approximation for a year

    def __str__(self):
        return self.value


class BusinessDayConvention(Enum):
    Following = "Following"
    ModifiedFollowing = "Modified Following"
    HalfMonthModifiedFollowing = "Half-Month Modified Following"
    Preceding = "Preceding"
    ModifiedPreceding = "Modified Preceding"
    Unadjusted = "Unadjusted"
    Nearest = "Nearest"

    def __str__(self):
        return self.value



In [99]:
class Calendar(ABC):
    ''' 
    abstract base class for calendar implementations
    '''

    def advance(self,
                start_date: date,
                period: int,
                time_unit: TimeUnit,
                convention: BusinessDayConvention,
                end_of_month: bool):
        ''' 
        Advances the given date as specified by the given period and
            returns the result.
        '''
        
        if start_date is None:
            raise ValueError("null date")

        if period == 0:
            return self.adjust(start_date, convention)

        d1 = start_date

        if time_unit == TimeUnit.Days:
            while period > 0:
                d1 += timedelta(days=1)
                while self.is_holiday(d1):
                    d1 += timedelta(days=1)
                period -= 1

            while period < 0:
                d1 -= timedelta(days=1)
                while self.is_holiday(d1):
                    d1 -= timedelta(days=1)
                period += 1

        elif time_unit == TimeUnit.Weeks:
            d1 += timedelta(weeks=period)
            d1 = self.adjust(d1, convention)

        else:  # Months or Years
            d1 += timedelta(days=period * time_unit.value)

            if end_of_month and self.is_end_of_month(start_date):
                return self.end_of_month(d1)

            d1 = self.adjust(d1, convention)

        return d1

    def adjust(self,
               d: date,
               c: BusinessDayConvention):
        ''' 
        Adjust date based on the business day convention
        '''
        if d is None:
            raise ValueError("null date")

        if c == BusinessDayConvention.Unadjusted:
            return d

        d1 = d

        if c in [BusinessDayConvention.Following, BusinessDayConvention.ModifiedFollowing, BusinessDayConvention.HalfMonthModifiedFollowing]:
            while self.is_holiday(d1):
                d1 += timedelta(days=1)

            if c in [BusinessDayConvention.ModifiedFollowing, BusinessDayConvention.HalfMonthModifiedFollowing]:
                if d1.month != d.month:
                    return self.adjust(d, BusinessDayConvention.Preceding)

                if c == BusinessDayConvention.HalfMonthModifiedFollowing and d.day <= 15 and d1.day > 15:
                    return self.adjust(d, BusinessDayConvention.Preceding)

        elif c in [BusinessDayConvention.Preceding, BusinessDayConvention.ModifiedPreceding]:
            while self.is_holiday(d1):
                d1 -= timedelta(days=1)

            if c == BusinessDayConvention.ModifiedPreceding and d1.month != d.month:
                return self.adjust(d, BusinessDayConvention.Following)

        elif c == BusinessDayConvention.Nearest:
            d2 = d
            while self.is_holiday(d1) and self.is_holiday(d2):
                d1 += timedelta(days=1)
                d2 -= timedelta(days=1)

            if self.is_holiday(d1):
                return d2
            else:
                return d1

        else:
            raise ValueError("unknown business-day convention")

        return d1
    
    def is_end_of_month(self, d: date):
        return d.month != self.adjust(d + timedelta(1).month)
    
    def is_weekend(self, d: date):
        return d.weekday() in [5,6]

    def end_of_month(self, d: date):
        pass

    def is_holiday(self, d: date):
        return not self.is_business_day(d)

    @abstractmethod
    def is_business_day(self, d: date):
        ''' 
        abstract method depending on different calendars
        '''
        pass


class TARGET(Calendar):
    def __init__(self) -> None:
        pass 

    def is_business_day(self, d: date):
        ''' 
        check se la data in input è un bd oppure no
        '''
        ny = d.day == 1 and d.month == 1
        em = d == easter(d.year) + timedelta(1)
        gf = d == easter(d.year) - timedelta(2)
        ld = d.day == 1 and d.month == 5
        c = d.day == 25 and d.month == 12
        cg = d.day == 26 and d.month == 12

        if self.is_weekend(d) or ny or gf or em or ld or c or cg:
            return False
        else:
            return True

In [100]:
d = date(2023,4,11)
calendar_test = TARGET()

In [103]:
calendar_test.is_business_day(d)

True

In [104]:
calendar_test.is_holiday(d)

False